In [4]:
import socket
import logging
import sys,re,urllib
import mechanize
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [5]:
browser = mechanize.Browser()
browser.set_handle_robots(False)
browser.open("http://volbrain.upv.es/index.php")
browser.select_form(nr=0)
print(browser)
#http://volbrain.upv.es/members.php
#http://volbrain.upv.es/uploadaction2.php 
browser["email"] = "romain_viard@yahoo.fr"
browser["password"] = "xIO52ts0UWm5"
browser.submit()
#browser.select_form(nr=0)

<Browser visiting http://volbrain.upv.es/index.php
 selected form:
 <login_form post http://volbrain.upv.es/login.php application/x-www-form-urlencoded
  <TextControl(email=)>
  <PasswordControl(password=)>
  <IgnoreControl(sub=<None>)>>
>


<response_seek_wrapper at 0x7f3a5000d5a8 whose wrapped object = <closeable_response at 0x7f3a41c5cf80 whose fp = <socket._fileobject object at 0x7f3a50040350>>>

In [6]:
#browser.open("http://volbrain.upv.es/uploadaction2.php")
browser.select_form(nr=0)
print(browser)
browser["volbrain_patientsage"]="18"
item = browser.find_control(name="volbrain_patientssex").get("Male")
item.selected = True
browser.form.add_file(open("/NAS/tupac/protocoles/healthy_volunteers/data/T02S02/20160418_145116WIP3DT1CLEARs201a1002.nii.gz"), 'text/plain', "/NAS/tupac/protocoles/healthy_volunteers/data/T02S02/20160418_145116WIP3DT1CLEARs201a1002.nii.gz", name='volbrain_t1_file')
print(browser)

<Browser visiting http://volbrain.upv.es/members.php
 selected form:
 <upload_form post http://volbrain.upv.es/uploadaction2.php multipart/form-data
  <HiddenControl(pipeline=2) (readonly)>
  <RadioControl(pipeline=[1, 3, 4])>
  <FileControl(volbrain_t1_file=<No files added>)>
  <SelectControl(volbrain_patientssex=[*--, Male, Female])>
  <TextControl(volbrain_patientsage=)>
  <IgnoreControl(button_volbrain=<None>)>
  <FileControl(ceres_t1_file=<No files added>)>
  <SelectControl(ceres_patientssex=[*--, Male, Female])>
  <TextControl(ceres_patientsage=)>
  <IgnoreControl(button_ceres=<None>)>
  <FileControl(lesionbrain_t1_file=<No files added>)>
  <SelectControl(lesionbrain_patientssex=[*--, Male, Female])>
  <TextControl(lesionbrain_patientsage=)>
  <IgnoreControl(button_lesionbrain=<None>)>
  <FileControl(lesionbrain_flair_file=<No files added>)>
  <RadioControl(modality=[1, 2, 3])>
  <FileControl(hips_mt1_file=<No files added>)>
  <SelectControl(hips_patientssex_t1=[*--, Male, Female

In [10]:
item = browser.find_control(name="button_volbrain")
print(item.attrs)
browser.submit()

browser.select_form(nr=0)
browser.viewing_html()
resp=browser.response()
html = resp.read()
soup = BeautifulSoup(html)
print(soup)

{u'type': u'button', u'name': u'button_volbrain', u'value': u'Submit', u'onclick': u"javascript:uploadData('volbrain');"}


<response_seek_wrapper at 0x7fef24262dd0 whose wrapped object = <closeable_response at 0x7fef240cd2d8 whose fp = <socket._fileobject object at 0x7fef240d6350>>>

# telecharge le fichier zip du fichier nifti en parametre 

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox
import datetime
not_fund=True
wanted_file="01022FZ20160323M003DT1_S002.nii.gz"
ladate=datetime.datetime.today().strftime('%Y-%m-%d')
myDict = {"romain_viard@yahoo.fr":"xIO52ts0UWm5","juliette.mitjans@yahoo.fr":"mdpvolbrain","rhumun@gmail.com":"EYyS9kx1UhWo","renaud.lopes@gmail.com":"Ci2c@chru"}

for cle,valeur in myDict.items() :
    if not_fund : 
        options = Options()
        options.add_argument("--headless")
        driver = Firefox(firefox_options=options)
        #driver = webdriver.Firefox()                
        driver.get("http://volbrain.upv.es/index.php")
        elem = driver.find_element_by_name("email")
        elem.send_keys(cle)
        elem = driver.find_element_by_name("password")
        elem.send_keys(valeur)
        elem = driver.find_element_by_name("sub")
        elem.click()
        elem = driver.find_element_by_name("job_list")
        html_tab_vol_brain=elem.get_attribute('innerHTML')
        txt_tab_vol_brain=elem.text
        #myIterator = re.finditer("http:\/\/files.volbrain.upv.es\/native.*?zip",tab_vol_brain,re.MULTILINE)
        for cpt in range(2,9999) :
            mytxtIterator = re.search(" "+str(cpt)+" ",txt_tab_vol_brain,re.MULTILINE)
            if mytxtIterator :
                myvar="loadJobList("+str(cpt)+")"
                driver.execute_script(myvar)
                elem = driver.find_element_by_name("job_list")
                html_tab_vol_brain=html_tab_vol_brain+elem.get_attribute('innerHTML')
                txt_tab_vol_brain=txt_tab_vol_brain+elem.text
            else :
                break
        myIterator = re.finditer("http:\/\/files.volbrain.upv.es\/native.*?zip",html_tab_vol_brain,re.MULTILINE)
        for it in myIterator :
            mystr=html_tab_vol_brain[it.start():it.end()]
            myfile = re.search("native_(.*nii.gz)",mystr,re.MULTILINE)
            if myfile.group(1) == wanted_file :
                #print urllib.urlopen(mystr).read() 
                !wget {mystr}
                not_fund=False
                break
        driver.close()
#iter_length = len(list(myIterator))
#print iter_length
#driver.execute_script("loadJobList(2)")

--2018-07-24 15:09:32--  http://files.volbrain.upv.es/native_01022FZ20160323M003DT1_S002.nii.gz_job89681250618195522.zip
Connexion à 10.50.2.2:3128… connecté.
requête Proxy transmise, en attente de la réponse… 200 OK
Taille : 3607937 (3,4M) [application/zip]
Enregistre : «native_01022FZ20160323M003DT1_S002.nii.gz_job89681250618195522.zip»

100%[======================================>] 3 607 937    137KB/s   ds 37s    

2018-07-24 15:10:10 (95,1 KB/s) - «native_01022FZ20160323M003DT1_S002.nii.gz_job89681250618195522.zip» enregistré [3607937/3607937]



BadStatusLine: ''

In [100]:
import glob,os,wget

Fileliste = glob.glob("/NAS/tupac/romain/test_docker/send_vb_docker/out/*.nii.gz.txt")
Fileliste.sort(key=os.path.getmtime)

myDict = {"romain_viard@yahoo.fr":"xIO52ts0UWm5","juliette.mitjans@yahoo.fr":"mdpvolbrain","rhumun@gmail.com":"EYyS9kx1UhWo","renaud.lopes@gmail.com":"Ci2c@chru"}

print "Nombre de fichiers dans le repertoire :"
print len(Fileliste)
print("\n".join(Fileliste))

for item in Fileliste :
	inputFile = open(item)
	idNumber = inputFile.readline()
	fullFileName = os.path.basename(item)
	print fullFileName[:-4]
	print idNumber

Nombre de fichiers dans le repertoire :
3
/NAS/tupac/romain/test_docker/send_vb_docker/out/T1_infile.nii.gz.txt
/NAS/tupac/romain/test_docker/send_vb_docker/out/01020VM20160120M003DT1_S002.nii.gz.txt
/NAS/tupac/romain/test_docker/send_vb_docker/out/01008GM20141022M003DT1_noCLEAR_GEO_S002.nii.gz.txt
T1_infile.nii.gz
3

01020VM20160120M003DT1_S002.nii.gz
2

01008GM20141022M003DT1_noCLEAR_GEO_S002.nii.gz
3


In [74]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox
import datetime
import random
ladate=datetime.datetime.today().strftime('%Y-%m-%d')
myDict = {"romain_viard@yahoo.fr":"xIO52ts0UWm5","juliette.mitjans@yahoo.fr":"mdpvolbrain","rhumun@gmail.com":"EYyS9kx1UhWo","renaud.lopes@gmail.com":"Ci2c@chru"}
ladate=datetime.datetime.today().strftime('%Y-%m-%d')

tmp_order=range(len(myDict))
random.shuffle(tmp_order)

for eval_order in tmp_order :
    cle = myDict.items()[eval_order][0]
    valeur = myDict.items()[eval_order][1]  
    options = Options()
    options.add_argument("--headless")
    driver = Firefox(firefox_options=options)
    #driver = webdriver.Firefox()
    driver.get("http://volbrain.upv.es/index.php")
    elem = driver.find_element_by_name("email")
    elem.send_keys(cle)
    elem = driver.find_element_by_name("password")
    elem.send_keys(valeur)
    elem = driver.find_element_by_name("sub")
    elem.click()
    elem = driver.find_element_by_name("job_list")
    html_tab_vol_brain=elem.get_attribute('innerHTML')
    txt_tab_vol_brain=elem.text
    for cpt in range(2,9999) :
        mytxtIterator = re.search(" "+str(cpt)+" ",txt_tab_vol_brain,re.MULTILINE)
        if mytxtIterator :
            myvar="loadJobList("+str(cpt)+")"
            driver.execute_script(myvar)
            elem = driver.find_element_by_name("job_list")
            html_tab_vol_brain=html_tab_vol_brain+elem.get_attribute('innerHTML')
            txt_tab_vol_brain=txt_tab_vol_brain+elem.text
        else :
            break
    cpt=0
    for m in re.finditer(" "+str(ladate)+" ",txt_tab_vol_brain):
        cpt += 1
    print cpt
    print cle
    print valeur 

3
renaud.lopes@gmail.com
Ci2c@chru
1
romain_viard@yahoo.fr
xIO52ts0UWm5
1
rhumun@gmail.com
EYyS9kx1UhWo
1
juliette.mitjans@yahoo.fr
mdpvolbrain


In [ ]:

        #myIterator = re.finditer("http:\/\/files.volbrain.upv.es\/native.*?zip",tab_vol_brain,re.MULTILINE)
        for cpt in range(2,9999) :
            mytxtIterator = re.search(" "+str(cpt)+" ",txt_tab_vol_brain,re.MULTILINE)
            if mytxtIterator :
                myvar="loadJobList("+str(cpt)+")"
                driver.execute_script(myvar)
                elem = driver.find_element_by_name("job_list")
                html_tab_vol_brain=html_tab_vol_brain+elem.get_attribute('innerHTML')
                txt_tab_vol_brain=txt_tab_vol_brain+elem.text
            else :
                break
        myIterator = re.finditer("http:\/\/files.volbrain.upv.es\/native.*?zip",html_tab_vol_brain,re.MULTILINE)
        for it in myIterator :
            mystr=html_tab_vol_brain[it.start():it.end()]
            myfile = re.search("native_(.*nii.gz)",mystr,re.MULTILINE)
            print myfile.group(1)
        driver.close()
#iter_length = len(list(myIterator))
#print iter_length
#driver.execute_script("loadJobList(2)")

In [259]:
for item in myList :
    if (len(item)>1) :
        mystr=item[2]
        myIterator = re.search("\/[^\/]+nii.gz$",mystr)
        print(mystr[myIterator.start()+1:myIterator.end()])

01022FZ20160323M003DT1_S002.nii.gz
01020VM20160120M003DT1_S002.nii.gz
01008GM20141022M003DT1_noCLEAR_GEO_S002.nii.gz
01007HC20140917M003DT1_noCLEAR_GEO_S002.nii.gz
01004DJ20140514M003DT1_noCLEAR_GEO_S001.nii.gz
01017TA20150923M003DT1_S002.nii.gz
01010VD20150311M003DT1_S003.nii.gz
01031FB20170118M003DT1_S002.nii.gz
01011BB20150506M003DT1_S002.nii.gz
01011BB20150909M003DT1_S002.nii.gz
01028SA20160525M003DT1_noCLEAR_GEO_S002.nii.gz
01025AF20160420M003DT1_S002.nii.gz
20151125_103646WIP3DT1ipatSENSEs201a1002.nii.gz
01014DE20151001M003DT1_S002.nii.gz
01002TM20140115M003DT1_noCLEAR_GEO_S002.nii.gz
01005LB20140618M003DT1_noCLEAR_GEO_S002.nii.gz
01023CF20160330M003DT1_S002.nii.gz
01012BB20150506M003DT1_S002.nii.gz
01030DM20170104M003DT1_S002.nii.gz
01009WF20141105M003DT1_noCLEAR_GEO_S002.nii.gz


In [263]:
myIterator = re.finditer("\/[^\/]+nii.gz$",html_tab_vol_brain,re.MULTILINE)
for it in myIterator :
    print(html_tab_vol_brain[it.start():it.end()])

In [10]:
#elems = driver.find_elements_by_class_name("par")
#for eleme_par in elems :
#    print(eleme_par.text)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import datetime

ladate=datetime.datetime.today().strftime('%Y-%m-%d')

driver = webdriver.Firefox()
driver.get("http://volbrain.upv.es/index.php")
elem = driver.find_element_by_name("email")
elem.send_keys("renaud.lopes@gmail.com")
elem = driver.find_element_by_name("password")
elem.send_keys("Ci2c@chru")
elem = driver.find_element_by_name("sub")
elem.click()

print(driver)
driver.execute_script("return 5")
driver.execute_script("loadJobList(2)")

<selenium.webdriver.firefox.webdriver.WebDriver (session="72c6ce29-67d6-4940-91a0-6ba0b2d3d091")>


# Permet d'envoyer un T1 associé à un age/sexe sur un compte  

In [192]:
import socket
import logging
import sys,re
import mechanize
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
#"romain_viard@yahoo.fr":"xIO52ts0UWm5"
#"juliette.mitjans@yahoo.fr":"mdpvolbrain"
#"renaud.lopes@gmail.com":"Ci2c@chu"
myDict = {"rhumun@gmail.com":"EYyS9kx1UhWo"}
#par defaut '''55''','''Male'''
myList=[['''58''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01022FZ/M00/NIFTI/3DT1_S002/01022FZ20160323M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01020VM/M00/NIFTI/3DT1_S002/01020VM20160120M003DT1_S002.nii.gz'''],
['''45''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01008GM/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01008GM20141022M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''67''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01007HC/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01007HC20140917M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01004DJ/M00/NIFTI/3DT1_noCLEAR_GEO_S001/01004DJ20140514M003DT1_noCLEAR_GEO_S001.nii.gz'''],
['''56''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01017TA/M00/NIFTI/3DT1_S002/01017TA20150923M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01010VD/M00/NIFTI/3DT1_S003/01010VD20150311M003DT1_S003.nii.gz'''],
['''54''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/001_031FB/M00/NIFTI/3DT1_S002/01031FB20170118M003DT1_S002.nii.gz'''],
['''53''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01011BB/M00/NIFTI/3DT1_S002/01011BB20150506M003DT1_S002.nii.gz'''],
['''53''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01011BB/M00/NIFTI/3DT1_rescan_S002/01011BB20150909M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01028SA/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01028SA20160525M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''64''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01025AF/M00/NIFTI/3DT1_S002/01025AF20160420M003DT1_S002.nii.gz'''],
['''50''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01018MT/20151125_103646WIP3DT1ipatSENSEs201a1002.nii.gz'''],
['''58''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01014DE/M00/NIFTI/3DT1_S002/01014DE20151001M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01002TM/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01002TM20140115M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01005LB/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01005LB20140618M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''59''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01023CF/M00/NIFTI/3DT1_S002/01023CF20160330M003DT1_S002.nii.gz'''],
['''58''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01012BB/M00/NIFTI/3DT1_S002/01012BB20150506M003DT1_S002.nii.gz'''],
['''66''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/001_030DM/NIFTI/3DT1_S002/01030DM20170104M003DT1_S002.nii.gz'''],
['''64''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01009WF/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01009WF20141105M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01013RP/M00/NIFTI/3DT1_S002/01013RP20150520M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01029LL/M00/NIFTI/3DT1_S002/01029LL20160706M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01021CT/M00/NIFTI/3DT1_S002/01021CT20160203M003DT1_S002.nii.gz'''],
['''61''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01003SJ/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01003SJ20140304M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''53''','''Feale''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01016BP/M00/NIFTI/3DT1_S002/01016BP20150916M003DT1_S002.nii.gz'''],
['''46''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01019LJ/M00/NIFTI/3DT1_S002/01019LJ20151209M003DT1_S002.nii.gz'''],
['''53''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01015HC/M00/NIFTI/3DT1_S002/01015HC20150902M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01006DF/M00/NIFTI/3DT1_noCLEAR_GEO_S002/01006DF20140827M003DT1_noCLEAR_GEO_S002.nii.gz'''],
['''59''','''Female''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01024DC/M00/NIFTI/3DT1_S002/01024DC20160406M003DT1_S002.nii.gz'''],
['''55''','''Male''','''/NAS/tupac/protocoles/predistim/CATI_data/Lille/convertData/01001SR/M00/NIFTI/3DT1_S002/01001SR20131120M003DT1_S002.nii.gz''']]
j=20
for cle,valeur in myDict.items() :
    for i in range(10) :
            item=myList[j]
            driver = webdriver.Firefox()
            driver.get("http://volbrain.upv.es/index.php")
            elem = driver.find_element_by_name("email")
            elem.send_keys(cle)
            elem = driver.find_element_by_name("password")
            elem.send_keys(valeur)
            elem = driver.find_element_by_name("sub")
            elem.click()
            elem = driver.find_element_by_id("pipeline1")
            elem.click()
            elem = driver.find_element_by_name("volbrain_t1_file")
            elem.send_keys(item.pop())
            elem = driver.find_element_by_name("volbrain_patientssex")
            elem.send_keys(item.pop())
            elem = driver.find_element_by_name("volbrain_patientsage")
            elem.send_keys(item.pop())
            elem = driver.find_element_by_name("button_volbrain")
            elem.click()
            print j
            if j>=(len(myList)-1) :
                break
            else :
                j=j+1
                time.sleep(5)

20
21
22
23
24
25
26
27
28
29
